<a href="https://colab.research.google.com/github/MonicaKim89/Review_analysis_text_mining/blob/main/%5B0524%5Dpreprocessing_pipe.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install konlpy
!pip install git+https://github.com/ssut/py-hanspell.git
!pip install transformers
!pip install sentencepiece
!pip install sacremoses
!pip install tensorflow_addons

In [4]:
import pandas as pd
from tqdm import tqdm
from konlpy.tag import Okt
okt = Okt()

from hanspell import spell_checker
import re
import joblib

from sklearn.feature_extraction.text import TfidfVectorizer
tf_vect = TfidfVectorizer()


### sent
import tensorflow as tf
import numpy as np
import pandas as pd
# from transformers import *
import json
import numpy as np
import pandas as pd
from tqdm import tqdm
import os
# import sentencepiece as spm

# import tensorflow_addons as tfa

In [8]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
%cd /content/drive/MyDrive/Projects/Derma/PIPE

/content/drive/MyDrive/Projects/Derma/PIPE


### **DATA**

In [46]:
train_data = pd.read_excel('..\\classification\\data\\[0523]class_df_v1.xlsx')
data_org = pd.read_excel("C:\\Users\\yukir\\Documents\\Monicas_workspace\\Derma\\important_data\\[0328]derma_final.xlsx")
# loaded_rf =  joblib.load('data/0524_RF_model_88.pkl')

### **BERT**

In [7]:
import logging
import os
import unicodedata
from shutil import copyfile

from transformers import PreTrainedTokenizer


logger = logging.getLogger(__name__)

VOCAB_FILES_NAMES = {"vocab_file": "tokenizer_78b3253a26.model",
                     "vocab_txt": "vocab.txt"}

PRETRAINED_VOCAB_FILES_MAP = {
    "vocab_file": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/tokenizer_78b3253a26.model",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/tokenizer_78b3253a26.model",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/tokenizer_78b3253a26.model"
    },
    "vocab_txt": {
        "monologg/kobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert/vocab.txt",
        "monologg/kobert-lm": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/kobert-lm/vocab.txt",
        "monologg/distilkobert": "https://s3.amazonaws.com/models.huggingface.co/bert/monologg/distilkobert/vocab.txt"
    }
}

PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES = {
    "monologg/kobert": 512,
    "monologg/kobert-lm": 512,
    "monologg/distilkobert": 512
}

PRETRAINED_INIT_CONFIGURATION = {
    "monologg/kobert": {"do_lower_case": False},
    "monologg/kobert-lm": {"do_lower_case": False},
    "monologg/distilkobert": {"do_lower_case": False}
}

SPIECE_UNDERLINE = u'▁'


class KoBertTokenizer(PreTrainedTokenizer):
    """
        SentencePiece based tokenizer. Peculiarities:
            - requires `SentencePiece <https://github.com/google/sentencepiece>`_
    """
    vocab_files_names = VOCAB_FILES_NAMES
    pretrained_vocab_files_map = PRETRAINED_VOCAB_FILES_MAP
    pretrained_init_configuration = PRETRAINED_INIT_CONFIGURATION
    max_model_input_sizes = PRETRAINED_POSITIONAL_EMBEDDINGS_SIZES

    def __init__(
            self,
            vocab_file,
            vocab_txt,
            do_lower_case=False,
            remove_space=True,
            keep_accents=False,
            unk_token="[UNK]",
            sep_token="[SEP]",
            pad_token="[PAD]",
            cls_token="[CLS]",
            mask_token="[MASK]",
            **kwargs):
        super().__init__(
            unk_token=unk_token,
            sep_token=sep_token,
            pad_token=pad_token,
            cls_token=cls_token,
            mask_token=mask_token,
            **kwargs
        )

        # Build vocab
        self.token2idx = dict()
        self.idx2token = []
        with open(vocab_txt, 'r', encoding='utf-8') as f:
            for idx, token in enumerate(f):
                token = token.strip()
                self.token2idx[token] = idx
                self.idx2token.append(token)

        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning("You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                           "pip install sentencepiece")

        self.do_lower_case = do_lower_case
        self.remove_space = remove_space
        self.keep_accents = keep_accents
        self.vocab_file = vocab_file
        self.vocab_txt = vocab_txt

        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(vocab_file)

    @property
    def vocab_size(self):
        return len(self.idx2token)

    def get_vocab(self):
        return dict(self.token2idx, **self.added_tokens_encoder)

    def __getstate__(self):
        state = self.__dict__.copy()
        state["sp_model"] = None
        return state

    def __setstate__(self, d):
        self.__dict__ = d
        try:
            import sentencepiece as spm
        except ImportError:
            logger.warning("You need to install SentencePiece to use KoBertTokenizer: https://github.com/google/sentencepiece"
                           "pip install sentencepiece")
        self.sp_model = spm.SentencePieceProcessor()
        self.sp_model.Load(self.vocab_file)

    def preprocess_text(self, inputs):
        if self.remove_space:
            outputs = " ".join(inputs.strip().split())
        else:
            outputs = inputs
        outputs = outputs.replace("``", '"').replace("''", '"')

        if not self.keep_accents:
            outputs = unicodedata.normalize('NFKD', outputs)
            outputs = "".join([c for c in outputs if not unicodedata.combining(c)])
        if self.do_lower_case:
            outputs = outputs.lower()

        return outputs

    def _tokenize(self, text, return_unicode=True, sample=False):
        """ Tokenize a string. """
        text = self.preprocess_text(text)

        if not sample:
            pieces = self.sp_model.EncodeAsPieces(text)
        else:
            pieces = self.sp_model.SampleEncodeAsPieces(text, 64, 0.1)
        new_pieces = []
        for piece in pieces:
            if len(piece) > 1 and piece[-1] == str(",") and piece[-2].isdigit():
                cur_pieces = self.sp_model.EncodeAsPieces(piece[:-1].replace(SPIECE_UNDERLINE, ""))
                if piece[0] != SPIECE_UNDERLINE and cur_pieces[0][0] == SPIECE_UNDERLINE:
                    if len(cur_pieces[0]) == 1:
                        cur_pieces = cur_pieces[1:]
                    else:
                        cur_pieces[0] = cur_pieces[0][1:]
                cur_pieces.append(piece[-1])
                new_pieces.extend(cur_pieces)
            else:
                new_pieces.append(piece)

        return new_pieces

    def _convert_token_to_id(self, token):
        """ Converts a token (str/unicode) in an id using the vocab. """
        return self.token2idx.get(token, self.token2idx[self.unk_token])

    def _convert_id_to_token(self, index, return_unicode=True):
        """Converts an index (integer) in a token (string/unicode) using the vocab."""
        return self.idx2token[index]

    def convert_tokens_to_string(self, tokens):
        """Converts a sequence of tokens (strings for sub-words) in a single string."""
        out_string = "".join(tokens).replace(SPIECE_UNDERLINE, " ").strip()
        return out_string

    def build_inputs_with_special_tokens(self, token_ids_0, token_ids_1=None):
        """
        Build model inputs from a sequence or a pair of sequence for sequence classification tasks
        by concatenating and adding special tokens.
        A KoBERT sequence has the following format:
            single sequence: [CLS] X [SEP]
            pair of sequences: [CLS] A [SEP] B [SEP]
        """
        if token_ids_1 is None:
            return [self.cls_token_id] + token_ids_0 + [self.sep_token_id]
        cls = [self.cls_token_id]
        sep = [self.sep_token_id]
        return cls + token_ids_0 + sep + token_ids_1 + sep

    def get_special_tokens_mask(self, token_ids_0, token_ids_1=None, already_has_special_tokens=False):
        """
        Retrieves sequence ids from a token list that has no special tokens added. This method is called when adding
        special tokens using the tokenizer ``prepare_for_model`` or ``encode_plus`` methods.
        Args:
            token_ids_0: list of ids (must not contain special tokens)
            token_ids_1: Optional list of ids (must not contain special tokens), necessary when fetching sequence ids
                for sequence pairs
            already_has_special_tokens: (default False) Set to True if the token list is already formated with
                special tokens for the model
        Returns:
            A list of integers in the range [0, 1]: 0 for a special token, 1 for a sequence token.
        """

        if already_has_special_tokens:
            if token_ids_1 is not None:
                raise ValueError(
                    "You should not supply a second sequence if the provided sequence of "
                    "ids is already formated with special tokens for the model."
                )
            return list(map(lambda x: 1 if x in [self.sep_token_id, self.cls_token_id] else 0, token_ids_0))

        if token_ids_1 is not None:
            return [1] + ([0] * len(token_ids_0)) + [1] + ([0] * len(token_ids_1)) + [1]
        return [1] + ([0] * len(token_ids_0)) + [1]

    def create_token_type_ids_from_sequences(self, token_ids_0, token_ids_1=None):
        """
        Creates a mask from the two sequences passed to be used in a sequence-pair classification task.
        A KoBERT sequence pair mask has the following format:
        0 0 0 0 0 0 0 0 0 0 1 1 1 1 1 1 1 1 1 1 1
        | first sequence    | second sequence
        if token_ids_1 is None, only returns the first portion of the mask (0's).
        """
        sep = [self.sep_token_id]
        cls = [self.cls_token_id]
        if token_ids_1 is None:
            return len(cls + token_ids_0 + sep) * [0]
        return len(cls + token_ids_0 + sep) * [0] + len(token_ids_1 + sep) * [1]

    def save_vocabulary(self, save_directory):
        """ Save the sentencepiece vocabulary (copy original file) and special tokens file
            to a directory.
        """
        if not os.path.isdir(save_directory):
            logger.error("Vocabulary path ({}) should be a directory".format(save_directory))
            return

        # 1. Save sentencepiece model
        out_vocab_model = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_file"])

        if os.path.abspath(self.vocab_file) != os.path.abspath(out_vocab_model):
            copyfile(self.vocab_file, out_vocab_model)

        # 2. Save vocab.txt
        index = 0
        out_vocab_txt = os.path.join(save_directory, VOCAB_FILES_NAMES["vocab_txt"])
        with open(out_vocab_txt, "w", encoding="utf-8") as writer:
            for token, token_index in sorted(self.token2idx.items(), key=lambda kv: kv[1]):
                if index != token_index:
                    logger.warning(
                        "Saving vocabulary to {}: vocabulary indices are not consecutive."
                        " Please check that the vocabulary is not corrupted!".format(out_vocab_txt)
                    )
                    index = token_index
                writer.write(token + "\n")
                index += 1

        return out_vocab_model, out_vocab_txt

In [31]:
tokenizer = KoBertTokenizer.from_pretrained('monologg/kobert')

loading file https://huggingface.co/monologg/kobert/resolve/main/tokenizer_78b3253a26.model from cache at /root/.cache/huggingface/transformers/7e55d7972628e6fc1babc614b5dd8bb43ab4f9d8541adc9fb1851112a7a7c5cc.4d2f4af7c2ca9df5b147978a95d38840e84801a378eee25756b008638e0bdc7f
loading file https://huggingface.co/monologg/kobert/resolve/main/vocab.txt from cache at /root/.cache/huggingface/transformers/efee434f5f4c5c89b5a7d8d5f30bbb0496f1540349fcfa21729cec5b96cfd2d1.719459e20bc981bc2093e859b02c3a3e51bab724d6b58927b23b512a3981229f
loading file https://huggingface.co/monologg/kobert/resolve/main/added_tokens.json from cache at None
loading file https://huggingface.co/monologg/kobert/resolve/main/special_tokens_map.json from cache at None
loading file https://huggingface.co/monologg/kobert/resolve/main/tokenizer_config.json from cache at /root/.cache/huggingface/transformers/d1c07e179f5e00959a3c8e4a150eaa4907dfe26544e4a71f2b0163982a476523.767d1b760a83978bae6c324157fad57ee513af333a7cea6986e8525

In [32]:
def convert_data(data_df):
    global tokenizer
    
    SEQ_LEN = 64 #SEQ_LEN : 버트에 들어갈 인풋의 길이
    
    tokens, masks, segments, targets = [], [], [], []
    
    for i in tqdm(range(len(data_df))):
        # token : 문장을 토큰화함
        token = tokenizer.encode(data_df[DATA_COLUMN][i], truncation=True, padding='max_length', max_length=SEQ_LEN)
       
        # 마스크는 토큰화한 문장에서 패딩이 아닌 부분은 1, 패딩인 부분은 0으로 통일
        num_zeros = token.count(0)
        mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros
        
        # 문장의 전후관계를 구분해주는 세그먼트는 문장이 1개밖에 없으므로 모두 0
        segment = [0]*SEQ_LEN

        # 버트 인풋으로 들어가는 token, mask, segment를 tokens, segments에 각각 저장
        tokens.append(token)
        masks.append(mask)
        segments.append(segment)
        
        # 정답(긍정 : 1 부정 0)을 targets 변수에 저장해 줌
        targets.append(data_df[LABEL_COLUMN][i])

    # tokens, masks, segments, 정답 변수 targets를 numpy array로 지정    
    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    targets = np.array(targets)

    return [tokens, masks, segments], targets

# 위에 정의한 convert_data 함수를 불러오는 함수를 정의
def load_data(pandas_dataframe):
    data_df = pandas_dataframe
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
    data_df[LABEL_COLUMN] = data_df[LABEL_COLUMN].astype(int)
    data_x, data_y = convert_data(data_df)
    return data_x, data_y

    
#------------------실제 테스트 할때 바꿀 것--------------------------------
SEQ_LEN = 8
BATCH_SIZE = 32
# 긍부정 문장을 포함하고 있는 칼럼
DATA_COLUMN = "sliced_reviews"
# 긍정인지 부정인지를 (1=긍정,0=부정) 포함하고 있는 칼럼
LABEL_COLUMN = "label"

def predict_convert_data(data_df):
    global tokenizer
    tokens, masks, segments = [], [], []
    
    for i in tqdm(range(len(data_df))):

        token = tokenizer.encode(data_df[DATA_COLUMN].tolist()[i], max_length=SEQ_LEN, truncation=True, padding='max_length')
        num_zeros = token.count(0)
        mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros
        segment = [0]*SEQ_LEN

        tokens.append(token)
        segments.append(segment)
        masks.append(mask)

    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    return [tokens, masks, segments]

# 위에 정의한 convert_data 함수를 불러오는 함수를 정의
def predict_load_data(pandas_dataframe):
    data_df = pandas_dataframe
    data_df[DATA_COLUMN] = data_df[DATA_COLUMN].astype(str)
    data_x = predict_convert_data(data_df)
    return data_x

In [60]:
def sentence_convert_data(data):
    global tokenizer
    tokens, masks, segments = [], [], []
    token = tokenizer.encode(data, max_length=SEQ_LEN, truncation=True, padding='max_length')
    
    num_zeros = token.count(0) 
    mask = [1]*(SEQ_LEN-num_zeros) + [0]*num_zeros 
    segment = [0]*SEQ_LEN

    tokens.append(token)
    segments.append(segment)
    masks.append(mask)

    tokens = np.array(tokens)
    masks = np.array(masks)
    segments = np.array(segments)
    return [tokens, masks, segments]


In [145]:
def derma_evaluation_predict(sentence):
    sent_class = []
    sent_score = []
    score_class = []

    data_x = sentence_convert_data(sentence)
    predict = sentiment_model.predict(data_x)
    predict_value = np.ravel(predict)
    predict_answer = np.round(predict_value,0).item()
    
    if predict_answer == 0:
        sent_class.append('부정')

        score = round(float(1-predict_value),2)
        sent_score.append(score)

    elif predict_answer == 1:
        sent_class.append('긍정')
        
        score = round(float(predict_value),2)
        sent_score.append(score)
        
    return sent_class, sent_score

### **SENTIMENTAL MODEL**

In [182]:
model = TFBertModel.from_pretrained("monologg/kobert", from_pt=True)
# 토큰 인풋, 마스크 인풋, 세그먼트 인풋 정의
token_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_word_ids')
mask_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_masks')
segment_inputs = tf.keras.layers.Input((SEQ_LEN,), dtype=tf.int32, name='input_segment')
# 인풋이 [토큰, 마스크, 세그먼트]인 모델 정의
bert_outputs = model([token_inputs, mask_inputs, segment_inputs])


bert_outputs = bert_outputs[1]
# Rectified Adam 옵티마이저 사용


# 총 batch size * 4 epoch = 2344 * 4
opt = tfa.optimizers.RectifiedAdam(lr=5.0e-5, total_steps = 2344*2, warmup_proportion=0.1, min_lr=1e-5, epsilon=1e-08, clipnorm=1.0)

sentiment_drop = tf.keras.layers.Dropout(0.5)(bert_outputs)
sentiment_first = tf.keras.layers.Dense(1, activation='sigmoid', kernel_initializer=tf.keras.initializers.TruncatedNormal(stddev=0.02))(sentiment_drop)
sentiment_model = tf.keras.Model([token_inputs, mask_inputs, segment_inputs], sentiment_first)
sentiment_model.compile(optimizer=opt, loss=tf.keras.losses.BinaryCrossentropy(), metrics = ['accuracy'])
sentiment_model.load_weights("/content/drive/MyDrive/Projects/Derma/Sentimental Analysis/sentimental models/weight[0523]sent_derma_v1.h5")


loading configuration file https://huggingface.co/monologg/kobert/resolve/main/config.json from cache at /root/.cache/huggingface/transformers/31dc8da633439f22ed80bede01f337996bc709eb8429f86f2b24e2103558b039.89a06cdfd16840fd89cc5c2493ef63cd0b6068e85f70ac988a3673e2722cab2e
Model config BertConfig {
  "architectures": [
    "BertModel"
  ],
  "attention_probs_dropout_prob": 0.1,
  "classifier_dropout": null,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "layer_norm_eps": 1e-12,
  "max_position_embeddings": 512,
  "model_type": "bert",
  "num_attention_heads": 12,
  "num_hidden_layers": 12,
  "pad_token_id": 1,
  "position_embedding_type": "absolute",
  "transformers_version": "4.19.2",
  "type_vocab_size": 2,
  "use_cache": true,
  "vocab_size": 8002
}

loading weights file https://huggingface.co/monologg/kobert/resolve/main/pytorch_model.bin from cache at /root/.cache/huggingface/transformers/9525d

### **SENTENCE PREPROCESSING**

In [14]:
def clean_text(text): 
    """ 한글, 영문, 숫자만 남기고 제거한다. 
    :param text: 
    :return: 
    """ 
    text = text.replace(".", " ").strip() 
    text = text.replace("·", " ").strip() 
    pattern = '[^ ㄱ-ㅣ가-힣|0-9|]+' 
    text = re.sub(pattern=pattern, repl='', string=text) 
    return text 

def stemming (text):
    morphs_list = []
    one_words = []
    result = []

    pos = okt.pos(text, join = False)
    # print(pos)
    try:
        for j in pos:
            if j.split('/')[1] == 'Noun':
                j = j.split('/')[0]
                morphs_list.append(j)
                        
            elif j.split('/')[1] =='Adjective':
                k = okt.morphs(j,  stem= True)
                k = k[0]
                morphs_list.append(k)
            elif j.split('/')[1] =='Verb':
                v = okt.morphs(j,  stem= True)
                v = v[0]
                morphs_list.append(v)

            elif j.split('/')[1] =='Adverb':
                ad = okt.morphs(j,  stem= True)
                v = v[0]
                morphs_list.append(ad)

        for i in morphs_list:
            if len(i) != 1:
                result.append(i)
        

        # for i in one_words:
        #     if i not in stopwords_kor:
        #         result.append(i)
    except:
        result.append(text)
        pass

    return result

def cleaned_sentence(x):
    cleaned_list = []
    str_x = str(x)
    pos = okt.pos(str_x, join = True)
    sent_ = str(pos)
    adj = sent_.split('/Adjective')
    for i in adj:
        v = i.split('/Verb') 
        for k in v:
            str_k = str(k)
            cln_k = clean_text(k)
            cln_k = cln_k.replace(' ', '')
            space = spell_checker.check(cln_k)
            text = space.checked
            cleaned_list.append(text)

    return cleaned_list

In [15]:
####
train_data = train_data.astype(str)
X_train = train_data['document']
tf_vect.fit(X_train)
    ####

TfidfVectorizer()

In [34]:
def class_sent(x):

    cleaned_list = []
    sent_class = []
    sent_score = []
    score_dim = []


    str_x = str(x)
    pos = okt.pos(str_x, join = True)
    sent_ = str(pos)
    adj = sent_.split('/Adjective')
    for i in adj:
        v = i.split('/Verb') 
        for k in v:
            str_k = str(k)
            cln_k = clean_text(k)
            cln_k = cln_k.replace(' ', '')
            space = spell_checker.check(cln_k)
            text = space.checked
            if len(text)<=4:
                pass
            else:
                cleaned_list.append(text)
            df_ = pd.DataFrame({'sliced_reviews':cleaned_list})
            X_test_tfidf_vect = tf_vect.transform(df_['sliced_reviews'])
            answer = loaded_rf.predict(X_test_tfidf_vect)

    #         df_['predict_class'] = answer
    # df_['predict_class'] = df_['predict_class'].replace("1","신뢰성")
    # df_['predict_class'] = df_['predict_class'].replace("2","반응성")
    # df_['predict_class'] = df_['predict_class'].replace("3","확신성")
    # df_['predict_class'] = df_['predict_class'].replace("4","공감성")
    # df_['predict_class'] = df_['predict_class'].replace("5","유형성")
    # df_['predict_class'] = df_['predict_class'].replace("6","일반문장")
    #         # print(f"학습 데이터 Text의 CountVectorizer Shape: {X_test_tfidf_vect.shape}")

    # ###### TFIDF
    # score_df = pd.DataFrame(df_['predict_class'].value_counts())

    #### SENTIMET ANALYSIS#####
    # pos_neg_list = []
    # sent_score_list =[]

    # pos_neg_list = []
    # sent_score_list =[]
    # # sent_class = []

    # for i in tqdm(df['sliced_reviews'].tolist()):
    #     pos_neg, sent_score = derma_evaluation_predict(i)
    #     pos_neg_list.append(pos_neg)
    #     sent_score_list.append(sent_score)
    #     # sent_class.append(sent_class)
    # df_['pos_neg'] = pos_neg_list
    # df_['sent_score'] = sent_score_list
    

    return df_, score_df

## **CLASSIFICATION**

In [36]:
####
train_data = train_data.astype(str)
X_train = train_data['document']
tf_vect.fit(X_train)
loaded_rf =  joblib.load('..\\classification\\workspace\\[0525]886_rf.pkl')
    ####

In [37]:
####
train_data = train_data.astype(str)
X_train = train_data['document']
tf_vect.fit(X_train)
    ####

TfidfVectorizer()

In [38]:
reviews_list = data_org[data_org['h']=='고운나라피부과의원'].reviews.tolist()

In [40]:
loaded_rf.predict(X_test_tfidf_vect)

NameError: name 'X_test_tfidf_vect' is not defined

In [ ]:
df, score_df = class_sent(reviews_list)

In [ ]:
df[20:40]

,sliced_reviews,predict_class,pos_neg,sent_score
20,이유에 대해서 설명을 잘,공감성,[부정],[0.92]
21,해주셨어요,공감성,[긍정],[0.52]
22,지인들에게 추천받아서,확신성,[부정],[0.81]
23,의사선생님과 진료 후에 상담실로 옮겨가서,공감성,[부정],[0.79]
24,치료 프로그램을 고르는,신뢰성,[부정],[0.98]
25,순서로 진행되었어요,확신성,[긍정],[0.94]
26,먼저 피지 조절하는,반응성,[부정],[0.94]
27,아쿠아 필 위주로 관리했습니다,일반문장,[부정],[0.57]
28,문신제거 문신한 곳일 부분만 지우고,일반문장,[긍정],[0.74]
29,여러 군데 알아보고,일반문장,[긍정],[0.91]


In [163]:
score_df

,predict_class
일반문장,34
확신성,24
공감성,7
신뢰성,7
반응성,1


### **Analyzing**

In [42]:
answer_list = []

for i in hos_1['sliced_reviews'].tolist():
    if len(i)>=60:
        answer_list.append('해결안됨')
    elif len(i)<=60:
        answer = derma_evaluation_predict(i)
        answer_list.append(answer) 
            

In [37]:
preds=sentiment_model.predict(test_set)

In [13]:
def clean_text(text): 
    """ 한글, 영문, 숫자만 남기고 제거한다. 
    :param text: 
    :return: 
    """ 
    text = text.replace(".", " ").strip() 
    text = text.replace("·", " ").strip() 
    pattern = '[^ ㄱ-ㅣ가-힣|0-9|]+' 
    text = re.sub(pattern=pattern, repl='', string=text) 
    return text 

def stemming (text):
    morphs_list = []
    one_words = []
    result = []

    pos = okt.pos(text, join = False)
    # print(pos)
    try:
        for j in pos:
            if j.split('/')[1] == 'Noun':
                j = j.split('/')[0]
                morphs_list.append(j)
                        
            elif j.split('/')[1] =='Adjective':
                k = okt.morphs(j,  stem= True)
                k = k[0]
                morphs_list.append(k)
            elif j.split('/')[1] =='Verb':
                v = okt.morphs(j,  stem= True)
                v = v[0]
                morphs_list.append(v)

            elif j.split('/')[1] =='Adverb':
                ad = okt.morphs(j,  stem= True)
                v = v[0]
                morphs_list.append(ad)

        for i in morphs_list:
            if len(i) != 1:
                result.append(i)
        

        # for i in one_words:
        #     if i not in stopwords_kor:
        #         result.append(i)
    except:
        result.append(text)
        pass

    return result

def cleaned_sentence(x):
    cleaned_list = []
    str_x = str(x)
    pos = okt.pos(str_x, join = True)
    sent_ = str(pos)
    adj = sent_.split('/Adjective')
    for i in adj:
        v = i.split('/Verb') 
        for k in v:
            str_k = str(k)
            cln_k = clean_text(k)
            cln_k = cln_k.replace(' ', '')
            space = spell_checker.check(cln_k)
            text = space.checked
            cleaned_list.append(text)

    return cleaned_list

In [26]:
/pwd

'/content/drive/MyDrive/Projects/Derma/PIPE/workspace'

In [27]:
%cd /content/drive/MyDrive/Projects/Derma/PIPE/workspace

/content/drive/MyDrive/Projects/Derma/PIPE/workspace


In [28]:
data_org = pd.read_excel('../data/[0328]derma_final.xlsx', index_col=0)
train_data = pd.read_excel("../data/[0523]class_df_v1.xlsx", index_col=0)

In [47]:
data = data_org.copy()
print(data.shape)
data.head(3)

(39216, 15)


,Unnamed: 0,h,adr,d,f,reviews,stars,treatments,money,비고,do,si,ro,adr_else,stemm
0,0,고운나라피부과의원,경남 김해시 내외중앙로 57 (내동),없음,"없음영수증 인증 3월 15일, 2022년 게시 ...",두피 가려움증으로 내원한 병원인데 진료 끝나고 두피 현미경사진을 무료로 찍어서 상담...,9.3,두피 가려움증,4900,리뷰없음,경남,김해시,내외중앙로,57,"['두피', '가려움증', '원한', '병원', '진료', '끝나다', '두피', ..."
1,1,고운나라피부과의원,경남 김해시 내외중앙로 57 (내동),없음,"없음영수증 인증 12월 20일, 2021년 게시 ...",대상포진으로 진료받았어요. 2주 이상 치료받고 약 먹고 주사 맞으니까 진통이 가라앉...,10,대상포진,4200,리뷰없음,경남,김해시,내외중앙로,57,"['대상포진', '진료', '받다', '치료받다', '먹다', '주사', '맞다',..."
2,2,고운나라피부과의원,경남 김해시 내외중앙로 57 (내동),없음,"없음영수증 인증 8월 11일, 2021년 게시 ...",여드름 고민이 있어서 방문했습니다. 피부전문의라서 증상이 나타난 이유에 대해서 설명...,10,관리 안됨,4000,리뷰없음,경남,김해시,내외중앙로,57,"['여드름', '고민', '방문', '피부', '전문의', '증상', '나타나다',..."


In [48]:
reviews = data[data['h']=='고운나라피부과의원'].reviews.tolist()

In [52]:
####
train_data = train_data.astype(str)
X_train = train_data['document']
tf_vect.fit(X_train)
loaded_rf =  joblib.load('..\\classification\\workspace\\0524_RF_model_88.pkl')
    ####

In [53]:
def cleaned_sentence(x):

    cleaned_list = []
    str_x = str(x)
    pos = okt.pos(str_x, join = True)
    sent_ = str(pos)
    adj = sent_.split('/Adjective')
    for i in adj:
        v = i.split('/Verb') 
        for k in v:
            str_k = str(k)
            cln_k = clean_text(k)
            cln_k = cln_k.replace(' ', '')
            space = spell_checker.check(cln_k)
            text = space.checked
            if len(text)<=4:
                pass
            else:
                cleaned_list.append(text)
            df_ = pd.DataFrame({'sliced_reviews':cleaned_list})
            X_test_tfidf_vect = tf_vect.transform(df_['sliced_reviews'])
            answer = loaded_rf.predict(X_test_tfidf_vect)

            df_['predict_class'] = answer
    df_['predict_class'] = df_['predict_class'].replace("1","신뢰성")
    df_['predict_class'] = df_['predict_class'].replace("2","반응성")
    df_['predict_class'] = df_['predict_class'].replace("3","확신성")
    df_['predict_class'] = df_['predict_class'].replace("4","공감성")
    df_['predict_class'] = df_['predict_class'].replace("5","유형성")
    df_['predict_class'] = df_['predict_class'].replace("6","일반문장")
            # print(f"학습 데이터 Text의 CountVectorizer Shape: {X_test_tfidf_vect.shape}")

    ###### TFIDF
    score_df = pd.DataFrame(df_['predict_class'].value_counts())
            

    return df_, score_df

In [54]:
hos_1, score_df = cleaned_sentence(reviews)
hos_1.head(20)

,sliced_reviews,predict_class
0,두피 가려움증으로 내원한,일반문장
1,병원인데 진료 끝나고,일반문장
2,두피 현미경사진을 무료로 찍어서,일반문장
3,상담해주셨어요,확신성
4,제 두피에 맞는,일반문장
5,샴푸 권하셔서,확신성
6,하나 사가지고,일반문장
7,샴푸 방법도 자세히,일반문장
8,알려주시고,확신성
9,친절하셨어요,공감성


In [34]:
score_df

,predict_class
일반문장,34
확신성,24
공감성,7
신뢰성,7
반응성,1


In [70]:
import random

work_list=[]
alist=[]                          # 뽑은 a를 넣어 중복 방지해주는 리스트         
for i in range(300):
  a = random.randint(1,38949)
  alist.append(a)      
  while a in alist :              # a가 이미 뽑은 리스트에 있을 때까지 다시 뽑자
    a = random.randint(1,38949)
    work_list.append(data_org['reviews'].tolist()[a])

len(work_list)

301

In [71]:
df, score_df = cleaned_sentence(work_list)

In [ ]:
df.to_excel('0525_derma_3.xlsx')

In [ ]:
test, score_df = cleaned_sentence('레이저토닝 친절하고 다른 거 권유 안 하고 필요한 것만 말해주셔서 좋고 직원분들 다 친절하세요. 시설도 좋고 하지만 예약해도 좀 대기가 있아요 당일예약도 되고 좋네요!! 친절해서 너무 좋습니다. 효과는 아직 일 주차라 모르겠어요.')

In [ ]:
test

,sliced_reviews,predict_class
0,레이저토닝 친절하고,공감성
1,다른 거 권유 안 하고,공감성
2,것만 말해주셔서,확신성
3,직원분들 다 친절하세요,확신성
4,시설도 좋고,유형성
5,하지만 예약해도,공감성
6,좀 대기가 있아요,반응성
7,당일예약도 되고,일반문장
8,너무 좋습니다,일반문장
9,효과는 아직 일 주차라 모르겠어요,유형성


In [ ]:
score_df

,predict_class
공감성,3
확신성,2
유형성,2
일반문장,2
반응성,1
